In [2]:
import mne
#import neurokit2 as nk2
from tqdm.notebook import tqdm


In [4]:
import sys
sys.path.append('../LaBraM-main')
import run_class_finetuning_sz_chlng_2025

In [5]:
TUEV_path = "/media/public/Datasets/TUEV/tuev/edf/processed_sz_chalenge_2025_full_file_sz_only"
Siena_path = "/media/public/Datasets/epilepsybenchmarks_chellenge/BIDS_Siena_to_labram_pkl/splited"
TUSZ_path = "/media/public/Datasets/epilepsybenchmarks_chellenge/tuh_train_preprocess_pkl/splited"
train_dataset, test_dataset, val_dataset = prepare_All_dataset_sz_chalenge_2025_full_files([Siena_path, TUSZ_path, TUEV_path])

NameError: name 'prepare_All_dataset_sz_chalenge_2025_full_files' is not defined

In [4]:
#raw = mne.io.read_raw_brainvision('/media/public/Datasets/MATBII/bids/sub-10/ses-2/eeg/sub-10_ses-2_task-MATBII_run-1_eeg.vhdr', preload=True)


Extracting parameters from /media/public/Datasets/MATBII/bids/sub-10/ses-2/eeg/sub-10_ses-2_task-MATBII_run-1_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 1615305  =      0.000 ...  3230.610 secs...


In [64]:
bands = {
    "Delta":[0.9, 4],
    "Theta":[4, 8],
    "Alpha":[8, 14],
    "Beta":[14, 25],
    "Gamma":[25, 40],
}
regions = {'Prefrontal': ['Fp1', 'Fp2'],
 'left_frontal': ['F7', 'F3', 'FC5'],
 'central_frontal': ['Fz', "AFz", 'FC1', 'FC2'],
 'right_frontal': ['F4', 'F8', 'FC6'],
 'left_temporal': ['T7', 'CP5', 'P7'],
 'right_temporal': ['T8', 'CP6', 'P8'],
 'central': ['C3', 'Cz', 'C4'],
 'parietal': ['P3', 'Pz',  'P4', 'CP1', 'CP2', 'CPz'],
 'occipital': ['O1', 'Oz', 'O2', "POz"]}


In [43]:
freq_list = [item for sublist in bands.values() for item in sublist]
epochs_length_sec = 5
epochs_overlap = 0.0
events = mne.make_fixed_length_events(raw, start=0, duration=epochs_length_sec, overlap=epochs_overlap)
epochs = mne.Epochs(raw, events, tmin=0, baseline=(0,0), tmax=epochs_length_sec)
# events.shape, epochs.get_data().shape
psds, freqs= mne.time_frequency.psd_array_multitaper(epochs.get_data(), sfreq=raw.info['sfreq'], fmin=min(freq_list), fmax=max(freq_list), bandwidth=None, adaptive=True, n_jobs=-1, verbose=0)

Not setting metadata
646 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 646 events and 2501 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.9s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.9s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    1.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.0s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done  10 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:    0.1

In [72]:
# morlet_wavelet = mne.time_frequency.tfr_morlet(epochs, n_cycles=, freqs=freq_list, n_jobs=-1, verbose=0)

In [73]:
# csd = mne.time_frequency.csd_array_multitaper(epochs.get_data(), sfreq=raw.info['sfreq'], fmin=min(freq_list), fmax=max(freq_list), bandwidth=None, adaptive=True, n_jobs=-1, verbose=0)

In [78]:
import pandas as pd
import numpy as np

def extract_band_region_features(tf_data, bands, regions, freqs, ch_names):
    # Convert freqs and ch_names to arrays for indexing
    freqs = np.array(freqs)
    ch_names = np.array(ch_names)

    feature_dict = {}

    for region_name, region_channels in regions.items():
        # Get indices of channels in this region
        ch_idx = [i for i, ch in enumerate(ch_names) if ch in region_channels]
        if not ch_idx:
            continue  # skip regions with no matching channels

        for band_name, (fmin, fmax) in bands.items():
            # Get indices of freqs in this band
            freq_idx = np.where((freqs >= fmin) & (freqs < fmax))[0]
            if len(freq_idx) == 0:
                continue  # skip empty bands

            # Average over selected channels and frequencies
            avg_power = tf_data[:, ch_idx][:, :, freq_idx].mean(axis=(1, 2))
            
            # Store feature with name like 'occipital_alpha'
            feature_name = f"{region_name.lower()}_{band_name.lower()}"
            feature_dict[feature_name] = avg_power

    # Convert to DataFrame
    df_features = pd.DataFrame(feature_dict)
    return df_features


df = extract_band_region_features(psds, bands, regions, freqs, raw.ch_names)

In [79]:
df

,prefrontal_delta,prefrontal_theta,prefrontal_alpha,prefrontal_beta,prefrontal_gamma,left_frontal_delta,left_frontal_theta,left_frontal_alpha,left_frontal_beta,left_frontal_gamma,...,parietal_delta,parietal_theta,parietal_alpha,parietal_beta,parietal_gamma,occipital_delta,occipital_theta,occipital_alpha,occipital_beta,occipital_gamma
0,294318.307644,115950.165730,36503.330498,6608.620842,1338.986159,75465.630533,23519.027241,7906.577626,1810.583150,520.167036,...,9500.682676,4647.756144,1792.423256,393.372100,249.476883,15360.504416,7775.379797,3138.857006,712.332355,526.432636
1,42178.295215,11996.048719,5162.522705,1871.414690,565.110987,26386.967604,5159.017216,1512.430074,605.532944,244.536456,...,4274.782276,1230.226666,796.571232,388.386597,158.301120,5373.681989,1906.338543,1477.511534,893.153948,378.125740
2,226681.629212,123021.622238,47188.108144,11679.044247,1368.469195,36211.383087,8478.905766,3498.930170,951.640785,198.333804,...,10114.348178,3388.972136,1571.036292,418.879284,188.229810,15915.229252,5616.897023,2668.163601,777.158898,440.292890
3,391024.817837,214933.862628,78076.780855,15346.420628,1538.271034,101655.323875,28020.835418,8237.699072,1759.657202,427.376371,...,11927.859287,4371.210731,2143.368737,469.063133,243.499213,20256.364623,7078.455410,3449.581993,982.121754,512.444401
4,349700.900403,169250.269152,64591.989324,9889.995430,2008.639141,98728.013882,28324.417627,6475.185071,1657.279823,692.105566,...,11349.232209,4933.718955,2370.081838,415.900302,237.285735,19133.616496,8585.266031,4291.443655,844.579664,470.092748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,262020.974825,130533.414511,42675.987512,6513.726732,3248.244881,39836.478249,7294.157811,3755.499840,1410.585860,971.264491,...,7227.916923,3549.683510,1323.790461,484.787613,172.358234,10793.391359,5149.115028,1904.308055,848.486861,352.443424
642,106895.147291,35024.292375,11255.466881,3274.703993,2296.669063,88751.115714,15912.132950,7201.195233,2095.931538,1315.453737,...,4760.690215,1289.430522,909.947746,1028.317426,252.820881,7222.324240,2187.734083,1686.733394,912.180160,431.644681
643,16847.591061,2907.597251,1820.007354,1342.092999,1479.738539,43840.853818,7091.141235,2746.414384,1120.264868,1160.012426,...,3339.774363,681.897249,684.905029,936.328880,153.775386,4769.701782,1200.990958,1153.246498,902.912404,254.324713
644,24780.016104,6204.075595,3060.859431,2875.298017,2420.574180,80727.684852,10770.435586,6270.135387,1770.719312,1328.225745,...,2659.614235,949.276559,635.835536,557.568498,180.203889,4071.950536,1747.847235,1141.941107,853.910404,266.049512
